In [1]:
import pandas as pd
import json

In [2]:
allprices = json.load(open("../_data/AllPrices.json"))
cards = pd.read_csv("../_data/cards.csv")

<ipython-input-2-2b8791a29cd9>:2: DtypeWarning: Columns (11,17,21,25,28,52,58,59,66,67,76,77,84,85) have mixed types. Specify dtype option on import or set low_memory=False.
  cards = pd.read_csv("../_data/cards.csv")


## Format Card Data

In [3]:
cards.columns

Index(['index', 'id', 'artist', 'asciiName', 'attractionLights',
       'availability', 'boosterTypes', 'borderColor', 'cardKingdomEtchedId',
       'cardKingdomFoilId', 'cardKingdomId', 'cardParts', 'cardsphereId',
       'colorIdentity', 'colorIndicator', 'colors', 'convertedManaCost',
       'duelDeck', 'edhrecRank', 'edhrecSaltiness', 'faceConvertedManaCost',
       'faceFlavorName', 'faceManaValue', 'faceName', 'finishes', 'flavorName',
       'flavorText', 'frameEffects', 'frameVersion', 'hand',
       'hasAlternativeDeckLimit', 'hasContentWarning', 'hasFoil', 'hasNonFoil',
       'isAlternative', 'isFullArt', 'isFunny', 'isOnlineOnly', 'isOversized',
       'isPromo', 'isRebalanced', 'isReprint', 'isReserved', 'isStarter',
       'isStorySpotlight', 'isTextless', 'isTimeshifted', 'keywords',
       'language', 'layout', 'leadershipSkills', 'life', 'loyalty', 'manaCost',
       'manaValue', 'mcmId', 'mcmMetaId', 'mtgArenaId', 'mtgjsonFoilVersionId',
       'mtgjsonNonFoilVersionI

In [4]:
cards = cards[['uuid', 'name', 'colorIdentity', 'colors', 'convertedManaCost',
                    'edhrecRank', 'edhrecSaltiness', 'isReserved', 'isReprint',
                    'keywords', 'language', 'loyalty', 'manaCost',
                    'manaValue', 'originalPrintings', 'originalReleaseDate', 'power', 'toughness',
                    'printings', 'rarity', 'setCode', 'type', 'types']].copy()

## Format Price Data

In [5]:
id_list = [i for i in allprices['data']]
len(id_list)

77055

In [6]:
price_list = []
for id in id_list:
  try:
    # only keep instances where there are paper cards sold at tcgplayer at retail price
    card_styles = allprices['data'][id].get('paper').get('tcgplayer').get('retail')
    for each_card in card_styles:
        for each_date in card_styles.get(each_card):
          price = card_styles.get(each_card).get(each_date)
          price_list.append( (id,each_card,each_date,price) )
  except:
    pass

In [7]:
price_df = pd.DataFrame(price_list, columns=['uuid', 'style', 'date','price'])

## Merge Card & Price Data

In [8]:
merged_df = price_df.merge(cards, on='uuid')

In [9]:
merged_df.sample(5)

,uuid,style,date,price,name,colorIdentity,colors,convertedManaCost,edhrecRank,edhrecSaltiness,...,manaValue,originalPrintings,originalReleaseDate,power,toughness,printings,rarity,setCode,type,types
1789716,4c538ca4-52a8-544a-9598-af957e05fce2,normal,2022-12-03,0.59,Ancient Amphitheater,"R,W",NaN,0.0,7373.0,0.04,...,0.0,NaN,NaN,NaN,NaN,"C15,CM2,LRW",rare,LRW,Land,Land
7283607,3f840a21-aa97-53a2-b814-dd66783da008,foil,2022-12-16,11.81,Wedding Announcement // Wedding Festivity,W,W,3.0,4759.0,NaN,...,3.0,NaN,NaN,NaN,NaN,"DBL,PRM,PVOW,VOW",rare,VOW,Enchantment,Enchantment
4074279,ad0ae266-c397-5bbd-8e25-531e3e6067c4,foil,2023-01-14,10.58,"Edric, Spymaster of Trest","G,U","G,U",3.0,2169.0,0.66,...,3.0,NaN,NaN,2,2,"C16,CM1,CMD,CNS,PCMD,PZ1,VMA",rare,CNS,Legendary Creature — Elf Rogue,Creature
1360645,39e91b03-f282-5565-a451-ee67fed2c2c5,foil,2022-12-26,0.95,"Keruga, the Macrosage","G,U","G,U",5.0,5596.0,0.21,...,5.0,NaN,NaN,5,4,"IKO,PIKO,PRM",rare,PIKO,Legendary Creature — Dinosaur Hippo,Creature
5233407,dd9b88b0-6de8-5268-abbe-cd3e5f9ba0c5,foil,2022-12-08,0.13,Saddleback Lagac,G,G,4.0,18824.0,0.29,...,4.0,NaN,NaN,3,1,"BBD,DDR,OGW",common,BBD,Creature — Lizard,Creature


## Output Data for Data Viz

In [10]:
# create a csv of the entire dataframe
merged_df.to_csv("../_data/card_prices_output.csv", index=False)

In [12]:
# without keeping card data, calculate only average price for creating histogram of average prices
avg_price = price_df[['uuid', 'style', 'price']].groupby(['uuid', 'style']).mean().round(2).reset_index()
avg_price

,uuid,style,price
0,00010d56-fe38-5e35-8aed-518019aa36a5,foil,8.69
1,0001e0d0-2dcd-5640-aadc-a84765cf5fc9,normal,5.51
2,0003caab-9ff5-5d1a-bc06-976dd0457f19,foil,4.93
3,0003caab-9ff5-5d1a-bc06-976dd0457f19,normal,0.19
4,0003d249-25d9-5223-af1e-1130f09622a7,foil,0.30
...,...,...,...
100815,fffa9242-6444-567c-a45b-da2b45249232,foil,0.47
100816,fffb659e-b3fa-5cd8-9423-fe5ac74248b5,normal,0.30
100817,fffbc95a-c4d1-56aa-8653-8a7c71fe19ce,normal,7.45
100818,fffc1305-a118-559b-9504-3d7b56ca0bde,normal,1.65


In [13]:
avg_price.to_csv('../_data/avg_price_output.csv', index=False)